<a href="https://colab.research.google.com/github/kuds/rl-atari-pong/blob/main/%5BAtari%20Pong%5D%20Multi-Agent%20Reinforcement%20Learning%20PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Atari Pong] Multi-Agent Reinforcement Learning PPO

In [1]:
!pip install swig

In [2]:
!pip install ray[rllib] gputil

In [3]:
!pip install gymnasium[atari]

In [4]:
!pip install pettingzoo[atari]

In [5]:
!pip install supersuit

In [6]:
!pip install autorom

In [7]:
!AutoROM --accept-license

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.10/dist-packages/AutoROM/roms
	/usr/local/lib/python3.10/dist-packages/multi_agent_ale_py/roms

Existing ROMs will be overwritten.


In [8]:
import os
import time
import torch
import numpy
import platform
import ray
import supersuit
from pettingzoo.atari import pong_v3
import torch
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.env import PettingZooEnv
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.torch_utils import FLOAT_MAX
from ray.tune.registry import register_env
from importlib.metadata import version

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:51: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are d

In [9]:
print(f"Python Version: {platform.python_version()}")
print(f"Torch Version: {version('torch')}")
print(f"Is Cuda Available: {torch.cuda.is_available()}")
print(f"Cuda Version: {torch.version.cuda}")
print(f"Gymnasium Version: {version('gymnasium')}")
print(f"Numpy Version: {version('numpy')}")
print(f"PettingZoo Version: {version('pettingzoo')}")
print(f"Ray Version: {version('ray')}")
print(f"ALE Version: {version('ale_py')}")
print(f"SuperSuit Version: {version('supersuit')}")

Python Version: 3.10.12
Torch Version: 2.4.0+cpu
Is Cuda Available: False
Cuda Version: None
Gymnasium Version: 0.28.1
Numpy Version: 1.26.4
PettingZoo Version: 1.24.3
Ray Version: 2.37.0
ALE Version: 0.8.1
SuperSuit Version: 3.9.3


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# class CNNModelV2(TorchModelV2, torch.nn.Module):
#     def __init__(self, obs_space, act_space, num_outputs, *args, **kwargs):
#         TorchModelV2.__init__(self, obs_space, act_space, num_outputs, *args, **kwargs)
#         torch.nn.Module.__init__(self)
#         self.model = torch.nn.Sequential(
#             torch.nn.Conv2d(3, 32, [8, 8], stride=(4, 4)),
#             torch.nn.ReLU(),
#             torch.nn.Conv2d(32, 64, [4, 4], stride=(2, 2)),
#             torch.nn.ReLU(),
#             torch.nn.Conv2d(64, 64, [3, 3], stride=(1, 1)),
#             torch.nn.ReLU(),
#             torch.nn.Flatten(),
#             (torch.nn.Linear(3136, 512)),
#             torch.nn.ReLU(),
#         )
#         self.policy_fn = torch.nn.Linear(512, num_outputs)
#         self.value_fn = torch.nn.Linear(512, 1)

#     def forward(self, input_dict, state, seq_lens):
#         model_out = self.model(input_dict["obs"].permute(0, 3, 1, 2))
#         self._value_out = self.value_fn(model_out)
#         return self.policy_fn(model_out), state

#     def value_function(self):
#         return self._value_out.flatten()


# def env_creator(args):
#     env = pong_v3.parallel_env(num_players=2)
#     env = supersuit.color_reduction_v0(env, mode="B")
#     env = supersuit.dtype_v0(env, "float32")
#     env = supersuit.resize_v1(env, x_size=84, y_size=84)
#     env = supersuit.normalize_obs_v0(env, env_min=0, env_max=1)
#     env = supersuit.frame_stack_v1(env, 4)
#     return env

# if __name__ == "__main__":
#     ray.init()

#     env_name = "pong_v3"

#     register_env(env_name, lambda config: ParallelPettingZooEnv(env_creator(config)))
#     ModelCatalog.register_custom_model("CNNModelV2", CNNModelV2)

#     config = (
#         PPOConfig()
#         .environment(env=env_name, clip_actions=True)
#         .rollouts(num_rollout_workers=1, rollout_fragment_length=128)
#         .training(
#             train_batch_size=512,
#             lr=2e-5,
#             gamma=0.99,
#             lambda_=0.9,
#             use_gae=True,
#             clip_param=0.4,
#             grad_clip=None,
#             entropy_coeff=0.1,
#             vf_loss_coeff=0.25,
#             sgd_minibatch_size=64,
#             num_sgd_iter=10,
#         )
#         .debugging(log_level="ERROR")
#         .framework(framework="torch")
#         .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
#     )

#     tune.run(
#         "PPO",
#         name="PPO",
#         stop={"timesteps_total": 5000000 if not os.environ.get("CI") else 50000},
#         checkpoint_freq=10,
#         storage_path="~/ray_results/" + env_name,
#         config=config.to_dict(),
#     )
#     ray.shutdown()

In [11]:
# import gymnasium as gym

# from ray import tune
# from ray.rllib.algorithms.ppo import PPOConfig
# from ray.rllib.connectors.env_to_module.frame_stacking import FrameStackingEnvToModule
# from ray.rllib.connectors.learner.frame_stacking import FrameStackingLearner

# from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
# from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
# from ray.rllib.utils.framework import try_import_torch
# from ray.rllib.utils.torch_utils import FLOAT_MAX
# from ray.rllib.env.wrappers.atari_wrappers import wrap_atari_for_new_api_stack
# from ray.rllib.utils.test_utils import add_rllib_example_script_args


# parser = add_rllib_example_script_args(
#     default_reward=float("inf"),
#     default_timesteps=3000000,
#     default_iters=100000000000,
# )
# parser.set_defaults(enable_new_api_stack=True)
# # Use `parser` to add your own custom command line options to this script
# # and (if needed) use their values toset up `config` below.
# args = parser.parse_args()


# def _make_env_to_module_connector(env):
#     return FrameStackingEnvToModule(num_frames=4)


# def _make_learner_connector(input_observation_space, input_action_space):
#     return FrameStackingLearner(num_frames=4)


# # Create a custom Atari setup (w/o the usual RLlib-hard-coded framestacking in it).
# # We would like our frame stacking connector to do this job.
# def _env_creator(cfg):
#     return wrap_atari_for_new_api_stack(
#         gym.make(args.env, **cfg, render_mode="rgb_array"),
#         # Perform frame-stacking through ConnectorV2 API.
#         framestack=None,
#     )


# tune.register_env("env", _env_creator)


# config = (
#     PPOConfig()
#     .environment(
#         "env",
#         env_config={
#             # Make analogous to old v4 + NoFrameskip.
#             "frameskip": 1,
#             "full_action_space": False,
#             "repeat_action_probability": 0.0,
#         },
#         clip_rewards=True,
#     )
#     .env_runners(
#         # num_envs_per_env_runner=5,  # 5 on old yaml example
#         env_to_module_connector=_make_env_to_module_connector,
#     )
#     .training(
#         learner_connector=_make_learner_connector,
#         train_batch_size_per_learner=4000,  # 5000 on old yaml example
#         minibatch_size=128,  # 500 on old yaml example
#         lambda_=0.95,
#         kl_coeff=0.5,
#         clip_param=0.1,
#         vf_clip_param=10.0,
#         entropy_coeff=0.01,
#         num_epochs=10,
#         lr=0.00015 * args.num_gpus,
#         grad_clip=100.0,
#         grad_clip_by="global_norm",
#     )
#     .rl_module(
#         model_config=DefaultModelConfig(
#             conv_filters=[[16, 4, 2], [32, 4, 2], [64, 4, 2], [128, 4, 2]],
#             conv_activation="relu",
#             head_fcnet_hiddens=[256],
#             vf_share_layers=True,
#         ),
#     )

#     # (
#     #     model_config_dict={"fcnet_hiddens": [32, 32]},
#     #     rl_module_spec=RLModuleSpec(module_class=DiscreteBCTorchModule),
#     # )
# )


# if __name__ == "__main__":
#     from ray.rllib.utils.test_utils import run_rllib_example_script_experiment

#     run_rllib_example_script_experiment(config, args=args)

In [12]:
import argparse
import json
import os
import pathlib
import time
import uuid

import gymnasium as gym
import numpy as np
import pandas as pd
import torch
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env.wrappers.atari_wrappers import wrap_deepmind
from ray.rllib.models import ModelCatalog
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override
from ray.tune.registry import register_env
from ray.rllib.utils.metrics import (
    ENV_RUNNER_RESULTS,
    EVALUATION_RESULTS,
    EPISODE_RETURN_MEAN,
    NUM_ENV_STEPS_SAMPLED_LIFETIME,
)
from torch import nn
from tqdm.rich import tqdm


def make_atari(env_config):
    env = gym.make(env_config["name"])
    env = wrap_deepmind(env, 84, True, True)
    return env


def linear_schedule(lr, n_iterations, iteration_steps):
    ts_lr = []
    ts = 0
    for iteration in range(1, n_iterations + 1):
        frac = 1.0 - (iteration - 1.0) / n_iterations
        ts_lr.append((ts, frac * lr))
        ts += iteration_steps
    return ts_lr


def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer


class Agent(TorchModelV2, nn.Module):
    def __init__(
        self, observation_space, action_space, num_outputs, model_config, name
    ):
        TorchModelV2.__init__(
            self, observation_space, action_space, num_outputs, model_config, name
        )
        nn.Module.__init__(self)

        self.network = nn.Sequential(
            layer_init(nn.Conv2d(4, 32, 8, stride=4)),
            nn.ReLU(),
            layer_init(nn.Conv2d(32, 64, 4, stride=2)),
            nn.ReLU(),
            layer_init(nn.Conv2d(64, 64, 3, stride=1)),
            nn.ReLU(),
            nn.Flatten(),
            layer_init(nn.Linear(64 * 7 * 7, 512)),
            nn.ReLU(),
        )
        self.actor = layer_init(nn.Linear(512, num_outputs), std=0.01)
        self.critic = layer_init(nn.Linear(512, 1), std=1)
        self.output = None

    @override(ModelV2)
    def forward(self, input_dict, state, seq_lens):
        x = input_dict["obs"] / 255.0
        x = torch.reshape(x, (-1, 4, 84, 84))
        self.output = self.network(x)
        return self.actor(self.output), []

    @override(ModelV2)
    def value_function(self):
        assert self.output is not None, "must call forward first!"
        return torch.reshape(self.critic(self.output), [-1])


def train_atari(args):
    total_timesteps = int(10e6)
    lr = 2.5e-4
    n_envs = 8
    n_steps = 128
    n_iterations = total_timesteps // (n_envs * n_steps)
    lr_schedule = linear_schedule(lr, n_iterations, n_steps * n_envs)

    ModelCatalog.register_custom_model("Agent", Agent)
    register_env(f"{args.env}", make_atari)

    ppo = (
        PPOConfig()
        .training(
            gamma=0.99,
            grad_clip_by="global_norm",
            train_batch_size=128 * 8,
            model={"custom_model": "Agent"},
            optimizer={"eps": 1e-5},
            lr_schedule=lr_schedule,
            use_critic=True,
            use_gae=True,
            lambda_=0.95,
            use_kl_loss=False,
            kl_coeff=0.0,  # not used
            kl_target=0.01,  # not used
            sgd_minibatch_size=256,
            num_sgd_iter=4,
            shuffle_sequences=True,
            vf_loss_coeff=0.5,
            entropy_coeff=0.01,
            entropy_coeff_schedule=None,
            clip_param=0.1,
            vf_clip_param=0.1,
            grad_clip=0.5,
        )
        .environment(
            env=f"{args.env}",
            env_config={"name": f"{args.env}NoFrameskip-v4"},
            render_env=False,
            clip_rewards=True,
            normalize_actions=False,
            clip_actions=False,
            is_atari=True,
        )
        .env_runners(
            num_env_runners=8,
            num_envs_per_env_runner=1,
            rollout_fragment_length=128,
            batch_mode="truncate_episodes",
            explore=True,
            exploration_config={"type": "StochasticSampling"},
            create_env_on_local_worker=False,
            preprocessor_pref=None,
            observation_filter="NoFilter",
        )
        .framework(framework="torch")
        .evaluation(
            evaluation_interval=None,
            evaluation_duration=100,
            evaluation_duration_unit="episodes",
            evaluation_config={
                "explore": True,
                "exploration_config": {"type": "StochasticSampling"},
            },
            evaluation_num_env_runners=1,
        )
        .debugging(seed=args.seed)
        # .resources(
        #     num_gpus=0.3,
        #     num_cpus_per_worker=1,
        #     num_gpus_per_worker=0,
        #     num_cpus_for_local_worker=1,
        # )
        .reporting(
            metrics_num_episodes_for_smoothing=100,
            min_train_timesteps_per_iteration=128 * 8,
            min_sample_timesteps_per_iteration=128 * 8,
        )
        .experimental(_disable_preprocessor_api=True, _enable_new_api_stack=False)
        .build()
    )

    # train
    start_time = time.time()
    progress_data = {"global_step": [], "mean_reward": []}
    for iteration in tqdm(range(1, n_iterations + 1)):
        result = ppo.train()
        #print(result)
        rewards = result[ENV_RUNNER_RESULTS]["hist_stats"]["episode_reward"]
        global_step = result["timesteps_total"]
        if len(rewards) > 100:
            rewards = rewards[-100:]
        mean_reward = np.nan if len(rewards) == 0 else float(np.mean(rewards))
        progress_data["global_step"].append(global_step)
        progress_data["mean_reward"].append(mean_reward)
    train_end_time = time.time()
    progress_df = pd.DataFrame(progress_data)
    progress_df.to_csv(os.path.join(args.path, "progress.csv"), index=False)

    # eval
    # there seems to be an issue where rllib does not follow specified evaluation episodes of 100
    # here we run evaluation until 100 episodes, store the final eval results as well as initial eval results
    initial_results = []
    results = []
    count = 0
    while len(results) < 100:
        result = ppo.evaluate()
        assert result["env_runner_results"]["episodes_this_iter"] == len(
            result["env_runner_results"]["hist_stats"]["episode_reward"]
        )
        results += result["env_runner_results"]["hist_stats"]["episode_reward"]
        if count == 0:
            initial_results += result["env_runner_results"]["hist_stats"][
                "episode_reward"
            ]
        count += 1
    eval_end_time = time.time()
    args.training_time_h = ((train_end_time - start_time) / 60) / 60
    args.total_time_h = ((eval_end_time - start_time) / 60) / 60
    args.eval_mean_reward = float(np.mean(results[:100]))
    args.initial_eval_mean_reward = float(np.mean(initial_results))
    args.initial_eval_episodes = len(initial_results)

    ppo.save(args.path)


# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument(
#         "-g",
#         "--gpu",
#         type=int,
#         help="Specify GPU index",
#         default=0,
#     )
#     parser.add_argument(
#         "-e",
#         "--env",
#         type=str,
#         help="Specify Atari environment w/o version",
#         default="Pong",
#     )
#     parser.add_argument(
#         "-t",
#         "--trials",
#         type=int,
#         help="Specify number of trials",
#         default=5,
#     )
#     args = parser.parse_args()
#     for _ in range(args.trials):
#         args.id = uuid.uuid4().hex
#         args.path = os.path.join("trials", "ppo", args.env, args.id)
#         args.seed = int(time.time())

#         # create dir
#         pathlib.Path(args.path).mkdir(parents=True, exist_ok=True)

#         # set gpu
#         os.environ["CUDA_VISIBLE_DEVICES"] = f"{args.gpu}"

#         train_atari(args)

#         # save trial info
#         with open(os.path.join(args.path, "info.json"), "w") as f:
#             json.dump(vars(args), f, indent=4)

In [ ]:
class Object(object):
    pass

args = Object()
args.env = "Pong"
args.id = uuid.uuid4().hex
args.path = os.path.join("trials", "ppo", args.env, args.id)
args.seed = int(time.time())
train_atari(args)

2024-10-20 17:56:00,262	WARNING deprecation.py:50 -- DeprecationWarning: `config.experimental(_enable_new_api_stack=...)` has been deprecated. Use `config.api_stack(enable_rl_module_and_learner=...)` instead. This will raise an error in the future!
2024-10-20 17:56:00,284	WARNING algorithm_config.py:4144 -- You have specified 1 evaluation workers, but your `evaluation_interval` is 0 or None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
/usr/local/lib/python3.10/dist-packages/ray/rllib/algorithms/algorithm.py:555: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/usr/local/lib/python3.10/dist-packages/ray/tune

Output()

<ipython-input-12-44cd737979c8>:174: TqdmExperimentalWarning: rich is experimental/alpha
  for iteration in tqdm(range(1, n_iterations + 1)):
2024-10-20 17:56:21,171	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


In [ ]:
import gymnasium as gym

from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.connectors.env_to_module.frame_stacking import FrameStackingEnvToModule
from ray.rllib.connectors.learner.frame_stacking import FrameStackingLearner

# Can import just yet because Ray RLLIB hasn't updated the pip packages yet
from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.env.wrappers.atari_wrappers import wrap_atari_for_new_api_stack
from ray.rllib.utils.test_utils import add_rllib_example_script_args


parser = add_rllib_example_script_args(
    default_reward=float("inf"),
    default_timesteps=3000000,
    default_iters=100000000000,
)
parser.set_defaults(enable_new_api_stack=True)
# Use `parser` to add your own custom command line options to this script
# and (if needed) use their values toset up `config` below.
args = parser.parse_args()


def _make_env_to_module_connector(env):
    return FrameStackingEnvToModule(num_frames=4)


def _make_learner_connector(input_observation_space, input_action_space):
    return FrameStackingLearner(num_frames=4)


# Create a custom Atari setup (w/o the usual RLlib-hard-coded framestacking in it).
# We would like our frame stacking connector to do this job.
def _env_creator(cfg):
    return wrap_atari_for_new_api_stack(
        gym.make(args.env, **cfg, render_mode="rgb_array"),
        # Perform frame-stacking through ConnectorV2 API.
        framestack=None,
    )


tune.register_env("env", _env_creator)


config = (
    PPOConfig()
    .environment(
        "env",
        env_config={
            # Make analogous to old v4 + NoFrameskip.
            "frameskip": 1,
            "full_action_space": False,
            "repeat_action_probability": 0.0,
        },
        clip_rewards=True,
    )
    .env_runners(
        # num_envs_per_env_runner=5,  # 5 on old yaml example
        env_to_module_connector=_make_env_to_module_connector,
    )
    .training(
        learner_connector=_make_learner_connector,
        train_batch_size_per_learner=4000,  # 5000 on old yaml example
        minibatch_size=128,  # 500 on old yaml example
        lambda_=0.95,
        kl_coeff=0.5,
        clip_param=0.1,
        vf_clip_param=10.0,
        entropy_coeff=0.01,
        num_epochs=10,
        lr=0.00015 * args.num_gpus,
        grad_clip=100.0,
        grad_clip_by="global_norm",
    )
    .rl_module(
        model_config=DefaultModelConfig(
            conv_filters=[[16, 4, 2], [32, 4, 2], [64, 4, 2], [128, 4, 2]],
            conv_activation="relu",
            head_fcnet_hiddens=[256],
            vf_share_layers=True,
        ),
    )
)


if __name__ == "__main__":
    from ray.rllib.utils.test_utils import run_rllib_example_script_experiment

    run_rllib_example_script_experiment(config, args=args)

In [ ]:
!pip install rllib/core/rl_module/default_model_config.py

In [ ]:
from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig